## Homework 08:  Parallel Programming 01

## Due Date: Apr 12, 2021, 04:00pm

#### Firstname Lastname: 

#### E-mail: 

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

For $r=0, 1, \dots, N - 1$, Process $r$ updates its buffer to the value received by $r-1$ (this should only be done for $r \geq 1$), then it squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ prints the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$. These are probably more than the available processes on your machine: you can use the option `--oversubscribe` in `mpirun` to let MPI run things anyway.



**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code. 



In [19]:
%%writefile sol08pr01.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

value = np.zeros(1)

if rank >= 1:
    #for r >= 1, receive the value from r - 1
    comm.Recv(value, source = rank - 1)
    #update its value
    value += rank ** 2
    
    #send the value to process r + 1, for process N-1, send the value to process 0
    if rank != size - 1:
        comm.Send(value, dest = rank + 1)
    else:
        comm.Send(value, dest = 0)
else:
    #for r = 0, first send its value to r = 1
    comm.Send(value, dest = 1)
    
    #receive the value from process N-1
    comm.Recv(value, source = size - 1)
    print("Process: {}, received value: {}".format(rank, value[0]))

Overwriting sol08pr01.py


In [20]:
#N = 10
!mpiexec -n 10 python sol08pr01.py

Process: 0, received value: 285.0


In [21]:
#N = 15
!mpiexec -n 15 python sol08pr01.py

Process: 0, received value: 1015.0


In [22]:
#N = 20
!mpiexec -n 20 python sol08pr01.py

Process: 0, received value: 2470.0


In [24]:
#N = 25
!mpiexec -n 25 python sol08pr01.py

Process: 0, received value: 4900.0


---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code.



In [56]:
%%writefile problem2.py

import sys
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

p = 1
x, y = float(sys.argv[1]), float(sys.argv[2])
buffer = np.array([x, y, p])


for i in range(5):
    if rank == 0:
        # multiply p by x
        buffer[2] *= buffer[0]
        
        # send the message to process 1 through channel 0
        comm.Send(buffer, dest = 1, tag = 0)
        
        # receive the message from process 1 through channel 1
        comm.Recv(buffer, source = 1, tag = 1)
        
        if i == 4:
            print(f"final value of p: {buffer[2]}")
    else:
        # receive the message from process 0 through channel 0
        comm.Recv(buffer, source = 0, tag = 0)
        
        # divide p by y
        buffer[2] /= buffer[1]
        
        # send the message to process 0 through channel 1
        comm.Send(buffer, dest = 0, tag = 1)
        

Overwriting problem2.py


In [57]:
# (2, 4)
!mpiexec -n 2 python problem2.py 2 4

final value of p: 0.03125


In [58]:
# (1, 3)
!mpiexec -n 2 python problem2.py 1 3

final value of p: 0.004115226337448559


In [59]:
# (5, 7)
!mpiexec -n 2 python problem2.py 5 7

final value of p: 0.1859344320818706


In [60]:
# (5, 10)
!mpiexec -n 2 python problem2.py 5 10

final value of p: 0.03125
